# Make Photographs Historical
There are 3 major steps in this notebook:
1. (optional) Convert images from HEIC image format to PNG
2. Preprocess images (so they are cropped and optionally downscaled)
3. Apply filters

The workflow uses 4 folders, where each transition/copy of images from one to another folder represents a major step.
By default, the folder names are:
1. 1_raw_input_images
2. 2_input_images
3. 3_preprocessed_images
4. 4_output_images

So if you want to skip steps 1 and 2, just place the images you want to apply filters for in folder 3_preprocessed_images.
A short explanation of the cells will show you if you need to execute the cell or you can skip it.

In [ ]:
############################################################
# MANDATORY
# Define folders and initiate folder structure
############################################################

import os
CWD = os.getcwd()
RAW_INPUT_IMAGES_PATH = f"{CWD}/data/1_raw_input_images"
INPUT_IMAGES_PATH = f"{CWD}/data/2_input_images"
PREPROCESSED_IMAGES_PATH = f"{CWD}/data/3_preprocessed_images"
OUTPUT_IMAGES_PATH = f"{CWD}/data/4_output_images"

!mkdir -p {INPUT_IMAGES_PATH}
!mkdir -p {RAW_INPUT_IMAGES_PATH}

!mkdir -p {PREPROCESSED_IMAGES_PATH}

#!rm -rf {OUTPUT_IMAGES_PATH}
!mkdir -p {OUTPUT_IMAGES_PATH}

In [ ]:
############################################################
# MANDATORY
# Set camera options, active camera and blender composition node mapping
############################################################

CAMERA_SETTINGS = {
    'hasselblad_500_cm': {
        'model_name': "Hasselblad 500 C/M",
        'aspect_ratio': { # 1:1 ratio
            'x': 9,
            'y': 9
        },
        'blender_node_group': "Camera: Hasselblad 500 C/M",
        'options': {}
    },
    'werra_mat': {
        'model_name': "Werra Mat",
        'aspect_ratio': { # 3:2 ratio
            'x': 24,
            'y': 36
        },
        'blender_node_group': "Camera: Werra Mat",
        'options': {}
    },
    'agfa_isolette': {
        'model_name': "Agfa Isolette",
        'aspect_ratio': { # 1:1 ratio
            'x': 6,
            'y': 6
        },
        'blender_node_group': "Camera: Agfar Isolette",
        'options': {}
    },
    'agfa_isolette_damaged': {
        'model_name': "Agfa Isolette",
        'aspect_ratio': { # 1:1 ratio
            'x': 6,
            'y': 6
        },
        'blender_node_group': "Camera: Agfar Isolette, with damage",
        'options': {}
    }
}

### SET FIELD HERE #######################################################################
ACTIVE_CAMERA_MODEL = CAMERA_SETTINGS["agfa_isolette_damaged"]
##########################################################################################


print(f"INFO: selected {ACTIVE_CAMERA_MODEL['model_name']} as the active camera")
print(f"----- STEP HAS FINISHED -----")

In [ ]:
############################################################
# STEP 1: Convert images from HEIC (RAW_INPUT_IMAGES_PATH) to PNG (INPUT_IMAGES_PATH)
############################################################

heic_files = [f for f in os.listdir(RAW_INPUT_IMAGES_PATH) if f.endswith('.HEIC') or f.endswith('.heic')]
print(f"INFO: found the following HEIC files {heic_files}")

import pyheif
from PIL import Image
import piexif

def extract_exif_metadata_from_file(file):
    for md_block in file.metadata:
        if md_block['type'] == 'Exif':
           return md_block['data']
    raise Exception("ERROR: no EXIF metadata found")
    return None

print(f"INFO: starting conversion from HEIC to PNG")
for filename in heic_files:
    print(f"\tINFO: converting {filename}")
    heic_file = pyheif.read(os.path.join(RAW_INPUT_IMAGES_PATH, filename))
    image = Image.frombytes(heic_file.mode, heic_file.size, heic_file.data)
    out_file_name = os.path.join(INPUT_IMAGES_PATH, os.path.splitext(filename)[0] + '.png')
    image.save(out_file_name)

    # Copy EXIF metadata from HEIC file to PNG file
    #raw_exif_bytes = extract_exif_metadata_from_file(heic_file)
    #exif_data = piexif.load(raw_exif_bytes)
    #print(exif_data)
    #piexif.insert(raw_exif_bytes, out_file_name)

# TODO: Does the metadata need to be copied for some purpose?
# I think yes, because we can then calculate delta between old camera and new camera and adjust settings accordingly

print(f"----- STEP HAS FINISHED -----")

In [ ]:
############################################################
# STEP 2: Preprocess images from (INPUT_IMAGES_PATH) to PNG (PREPROCESSED_IMAGES_PATH)
############################################################

DOWNSCALING_FACTOR = 2

from PIL import Image

def crop_to_aspect_ratio(image, w, h):
    # Calculate the target aspect ratio
    target_aspect_ratio = w / h
    
    # Get the original dimensions
    original_width, original_height = image.size
    
    # Determine the dimensions for the new aspect ratio
    new_width = original_width
    new_height = int(new_width / target_aspect_ratio)
    
    # If the calculated height is greater than the original, recalculate the width instead
    if new_height > original_height:
        new_height = original_height
        new_width = int(new_height * target_aspect_ratio)
    
    # Calculate the cropping area
    left = (original_width - new_width) / 2
    top = (original_height - new_height) / 2
    right = (original_width + new_width) / 2
    bottom = (original_height + new_height) / 2
    
    # Crop the image to the new aspect ratio
    cropped_image = image.crop((left, top, right, bottom))
    
    return cropped_image

    # Save or display the cropped image
    #cropped_image.save(output_path)
    #cropped_image.show()


input_files = [f for f in os.listdir(INPUT_IMAGES_PATH) if f.endswith('.PNG') or f.endswith('.png')]
print(f"INFO: found input files {input_files}")

print(f"INFO: starting to apply image filters for camera {ACTIVE_CAMERA_MODEL['model_name']}")
for filename in input_files:
    in_file_path = f"{INPUT_IMAGES_PATH}/{str(filename)}"
    image = Image.open(in_file_path)
    
    print(f"\tINFO: image {filename}")
    print(f"\tINFO: step 1: cropping image")
    cropped_image = crop_to_aspect_ratio(image, ACTIVE_CAMERA_MODEL['aspect_ratio']['x'], ACTIVE_CAMERA_MODEL['aspect_ratio']['y'])

    # Downscale image
    if DOWNSCALING_FACTOR and DOWNSCALING_FACTOR != 1:
        cropped_image_w, cropped_image_h = cropped_image.size
        if DOWNSCALING_FACTOR > 1:
            new_w = cropped_image_w / DOWNSCALING_FACTOR
            new_h = cropped_image_h / DOWNSCALING_FACTOR
        else:
            new_w = cropped_image_w * DOWNSCALING_FACTOR
            new_h = cropped_image_h * DOWNSCALING_FACTOR

        new_size = (int(new_w), int(new_h))
        print(f"\tINFO: step 2: downscaling image from {cropped_image_w}/{cropped_image_h} to {new_size[0]}/{new_size[0]}")
        cropped_image = cropped_image.resize(new_size)
    
    out_file_name = os.path.join(PREPROCESSED_IMAGES_PATH, os.path.splitext(filename)[0] + '.png')
    cropped_image.save(out_file_name)


print(f"----- STEP HAS FINISHED -----")

In [ ]:
############################################################
# STEP 3: Apply image filters
############################################################

BLENDER_SCRIPT_PATH = f"{CWD}/blender/scripts/batch_composite.py"
BLENDER_SCENE_PATH = f"{CWD}/blender/main.blend"
ACTIVE_FILTER_NAME = ACTIVE_CAMERA_MODEL['blender_node_group']

# if the process keeps dying, its probably because jupyter kills the process or times out! In that case just use the command:
print(f'blender "{BLENDER_SCENE_PATH}" -b --python "{BLENDER_SCRIPT_PATH}" -- "{ACTIVE_FILTER_NAME}" "{PREPROCESSED_IMAGES_PATH}" "{OUTPUT_IMAGES_PATH}"')

# start blender with a scene in background and execute script
# call like that 'python test.py -- <FILTER_NAME> <INPUT_PATH> <OUTPUT_PATH>
! blender "{BLENDER_SCENE_PATH}" -b --python "{BLENDER_SCRIPT_PATH}" -- "{ACTIVE_FILTER_NAME}" "{PREPROCESSED_IMAGES_PATH}" "{OUTPUT_IMAGES_PATH}"